# Prisoner's Dilemma Experiments

This notebook demonstrates how to run professional Prisoner's Dilemma experiments using the Concordia framework. We'll set up the environment, configure agents, and run multiple trials with proper logging and analysis.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import List, Dict, Any

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

# Import Concordia components
from concordia import language_model
from concordia import measurements
from concordia.associative_memory import importance_function
from concordia.utils import measurements as measurements_lib
from concordia.typing import scene as scene_lib
from concordia.typing import agent as agent_lib

# Import our custom environment
from prisoners_dilemma.environment import Simulation
from prisoners_dilemma.environment import PDPayoff

## 1. Initialize Language Models and Embedder

First, we'll set up the language models for both the game master and agents, along with the embedder for memory operations.

In [ ]:
# Initialize language models
model = language_model.LanguageModel()  # Replace with your preferred model

# Initialize embedder (using sentence-transformers as an example)
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Create a wrapper function for the embedder
def embed_text(text: str) -> np.ndarray:
    return embedder.encode(text)

# Initialize measurements
measurements = measurements_lib.Measurements()

## 2. Configure Experiment Parameters

Set up the parameters for our experiment, including the number of trials, rounds per trial, and agent configurations.

In [ ]:
# Experiment configuration
EXPERIMENT_CONFIG = {
    'num_trials': 10,
    'rounds_per_trial': 5,
    'agent_types': [
        'analytical',
        'cooperative',
        'skeptical'
    ],
    'random_seed': 42,
    'output_dir': 'experiment_results'
}

# Create output directory if it doesn't exist
os.makedirs(EXPERIMENT_CONFIG['output_dir'], exist_ok=True)

# Initialize results storage
results = {
    'trial': [],
    'round': [],
    'player': [],
    'action': [],
    'score': [],
    'total_score': [],
    'cooperation_rate': []
}

## 3. Run Experiments

Execute multiple trials of the Prisoner's Dilemma experiment and collect results.

In [ ]:
def run_experiment_trial(trial_num: int) -> Dict[str, Any]:
    """Run a single trial of the Prisoner's Dilemma experiment."""
    
    # Initialize simulation with unique seed for each trial
    seed = EXPERIMENT_CONFIG['random_seed'] + trial_num
    simulation = Simulation(
        gm_model=model,
        agent_model=model,
        embedder=embed_text,
        measurements=measurements,
        seed=seed
    )
    
    # Run simulation
    outcome, html_log = simulation()
    
    # Extract results
    trial_results = {
        'trial': trial_num,
        'outcome': outcome,
        'html_log': html_log
    }
    
    return trial_results


trial_results = []
for trial in range(EXPERIMENT_CONFIG['num_trials']):
    print(f"Running trial {trial + 1}/{EXPERIMENT_CONFIG['num_trials']}")
    result = run_experiment_trial(trial)
    trial_results.append(result)
    
    # Save HTML log for this trial
    log_path = os.path.join(
        EXPERIMENT_CONFIG['output_dir'],
        f'trial_{trial + 1}_log.html'
    )
    with open(log_path, 'w') as f:
        f.write(result['html_log'])

## 4. Analyze Results

Process and visualize the experiment results.

In [ ]:
def analyze_results(trial_results: List[Dict[str, Any]]) -> pd.DataFrame:
    """Convert trial results into a pandas DataFrame for analysis."""
    
    data = []
    for trial_num, result in enumerate(trial_results):
        outcome = result['outcome']
        
        # Extract scores and metadata
        for player_name, score in outcome.agent_scores.items():
            data.append({
                'trial': trial_num + 1,
                'player': player_name,
                'final_score': score,
                'agent_type': next(
                    agent_type for agent_type in EXPERIMENT_CONFIG['agent_types']
                    if agent_type in player_name.lower()
                )
            })
    
    return pd.DataFrame(data)

# Create DataFrame
results_df = analyze_results(trial_results)

# Display summary statistics
print("\nSummary Statistics:")
print(results_df.groupby('agent_type')['final_score'].agg(['mean', 'std', 'min', 'max']))

# Save results to CSV
results_df.to_csv(
    os.path.join(EXPERIMENT_CONFIG['output_dir'], 'experiment_results.csv'),
    index=False
)

## 5. Visualize Results

Create visualizations to better understand the experiment outcomes.

In [ ]:
def plot_results(df: pd.DataFrame):
    """Create visualizations of the experiment results."""
    
    # Set style
    plt.style.use('seaborn')
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot 1: Box plot of scores by agent type
    sns.boxplot(data=df, x='agent_type', y='final_score', ax=ax1)
    ax1.set_title('Score Distribution by Agent Type')
    ax1.set_xlabel('Agent Type')
    ax1.set_ylabel('Final Score')
    
    # Plot 2: Score trends across trials
    for agent_type in df['agent_type'].unique():
        agent_data = df[df['agent_type'] == agent_type]
        ax2.plot(agent_data['trial'], agent_data['final_score'], 
                label=agent_type, marker='o')
    
    ax2.set_title('Score Trends Across Trials')
    ax2.set_xlabel('Trial Number')
    ax2.set_ylabel('Final Score')
    ax2.legend()
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(os.path.join(EXPERIMENT_CONFIG['output_dir'], 'results_visualization.png'))
    plt.close()

# Generate visualizations
plot_results(results_df)

## 6. Generate Experiment Report

Create a comprehensive report of the experiment results.

In [ ]:
def generate_report(df: pd.DataFrame):
    """Generate a comprehensive report of the experiment results."""
    
    report = [
        "# Prisoner's Dilemma Experiment Report\n",
        f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n",
        "## Experiment Configuration\n",
        f"- Number of Trials: {EXPERIMENT_CONFIG['num_trials']}",
        f"- Rounds per Trial: {EXPERIMENT_CONFIG['rounds_per_trial']}",
        f"- Agent Types: {', '.join(EXPERIMENT_CONFIG['agent_types'])}",
        f"- Random Seed: {EXPERIMENT_CONFIG['random_seed']}\n",
        "## Results Summary\n",
        "### Overall Statistics\n",
        df.groupby('agent_type')['final_score'].agg(['mean', 'std', 'min', 'max']).to_markdown(),
        "\n### Detailed Results\n",
        df.to_markdown()
    ]
    
    # Save report
    report_path = os.path.join(EXPERIMENT_CONFIG['output_dir'], 'experiment_report.md')
    with open(report_path, 'w') as f:
        f.write('\n'.join(report))
    
    return report_path

# Generate and display report path
report_path = generate_report(results_df)
print(f"Experiment report saved to: {report_path}")